<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_5_LangChain%E1%84%92%E1%85%AA%E1%86%AF%E1%84%8B%E1%85%AD%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain을 활용한 RAG 실습

LangChain은 LLM에 입력되는 프롬프트에 다양한 엔지니어링 요소(프롬프트 템플릿, 체인, 메모리, 툴 호출 등)를 결합하여 하나의 완성된 애플리케이션 형태로 구성할 수 있는 단계별 워크플로우 프레임워크입니다.

또한 OpenAI, Hugging Face 등 주요 LLM 플랫폼과 손쉽게 연동할 수 있어 확장성이 뛰어납니다.

### 필요 라이브러리 설치

In [ ]:
!pip install langchain langchain-core langchain_community faiss-cpu tiktoken langchain-openai langchain-text-splitters datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## LangChain 프롬프트 엔지니어링

LangChain은 입력된 프롬프트를 탬플릿화 시키고 불러온 LLM 모델과 연결하여 단계별 워크플로우를 하나의 프로세스로 합쳐 줍니다.  



**단계별 워크플로우**:

    1. LLM 모델로드
    2. 프롬프트 템플릿 생성
    3. LLM 모델과 템플릿을 연결(Chains)
    4. 사용자 입력을 받아 모델에 입력
    5. 모델의 결과를 후처리

In [ ]:
from getpass import getpass
from langchain_openai import ChatOpenAI
import os

# OpenAI API key
openai_api_key = getpass("Enter your OpenAI API key: ")

# API키 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API key: ··········


In [ ]:
# 모델 로드
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

### 프롬프트 템플릿 생성

`langchain_core.prompts` 모듈은 `PromptTemplate` 객체를 제공하여 사용자가 입력한 프롬프트에 적절한 템플릿을 덮어 씌워 모델에 입력할수 있게 도와줍니다.  
이때 `PromptTemplate`에 입력되는 문자열 안에 `{변수이름}`형식으로 수정 가능한 변수를 할당할 수 있습니다.

In [ ]:
from langchain_core.prompts import PromptTemplate

# 텍스트 요약 템플릿
summary_template = """
당신은 전문 요약가입니다.
아래 주어진 텍스트를 50단어 이내로 간결하게 요약해 주세요.

텍스트:
{text_to_summarize}

요약:
"""
# PromptTemplates 템플릿 구성
summary_prompt = PromptTemplate.from_template(summary_template)

### LLM - Prompt 체인 만들기 (LCEL 방식)

LangChain에서는 기존의 LLMChain.run() 방식 대신,
**LangChain Expression Language(LCEL)**을 사용해
프롬프트 → LLM → 출력 파싱 과정을 하나의 체인으로 간단히 구성할 수 있습니다.

summary_prompt | llm | StrOutputParser() 형태로 체인을 만들고,
.invoke()에 사용자 입력을 넣으면 최종 생성 텍스트가 반환됩니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Using LangChain Expression Language (LCEL) to create the chain
summary_chain = summary_prompt | llm | StrOutputParser()

# 사용자 프롬프트
text_to_summarize = """
[데일리안 = 조인영 기자] 우리나라 앱 개발자가 앱 마켓사업자에게서 경험하는 주요 불공정 사례는 심사 지연과 등록 거부이며, 앱 내 결제(인앱결제)의 가장 큰 문제점은 ‘과도한 수수료’라고 인식하고 있는 것으로 나타났다.

방송통신위원회와 한국인터넷진흥원은 11일 '전기통신사업법' 제22조의9(앱 마켓사업자의 의무 및 실태조사)에 따른 ‘2024년도 앱 마켓 실태조사 결과’를 발표했다.

2023년도 국내 ‘앱 마켓’ 규모는 거래액 기준으로 8조1952억원으로 2022년 8조 7598억원 대비 6.4% 감소했다

각 사업자별로 보면, 애플 앱스토어(10.1%)와 삼성전자 갤럭시스토어(6.3%)는 전년 대비 매출이 증가했으며, 구글 플레이(△10.1%)와 원스토어(△21.6%)는 감소했다.

4개 앱 마켓사업자의 거래액 대비 수수료 비중은 약 14~26% 수준이며, 앱 등록 매출의 경우 애플 앱스토어는 약 9.4% 증가했고 구글 플레이는 약 12.9% 감소했다.

또한 국내 앱 마켓에 등록된 전체 앱 수는 전년 대비 0.1% 증가한 531만8182개(각 앱 마켓별 중복 포함), 앱 개발자 수는 전년 대비 0.65% 적은 163만6655명(각 앱 마켓별 중복 포함)으로 나타났다.

분야별 앱 등록 비중은 사진‧도구(26.1%), 라이프스타일(15.6%), 미디어‧출판(14.5%) 관련 앱이 전체의 56.2%를 차지했다.

국내 앱 개발자가 이용하는 앱 마켓은 구글 플레이(96.4%), 애플 앱스토어(71.3%) 순(중복포함)이며, 매출액 비중도 구글 플레이가 67.5%로 가장 높았다. 그 를 애플 앱스토어(28.2%), 원스토어(2.9%), 갤럭시스토어(1.5%)가 따랐다.

앱 개발자가 느끼는 주요 불공정 사례로는 앱 심사지연 경험(애플 앱스토어 6.8%, 구글 플레이 26.2%)이 가장 높았으며, 앱 등록 거부 경험(애플 20%, 구글 3%)과 앱 삭제 경험(구글 8.2%, 애플 3.2%)이 그 뒤를 이었다.

앱 내 결제의 가장 큰 문제점은 ‘과도한 수수료’라고 답한 앱 개발자가 0.4%로 가장 높게 나타났으며, 다음으로 ‘환불 등 수익 정산의 불명확함’(11.6%), ‘결제 수단 선택 제한’(8.9%) 순으로 조사됐다.

앱 개발자가 느낀 앱 최초 등록 과정상의 어려움으로는 ‘심사 기준이 확하지 않음’(구글 플레이 29.8%, 애플 앱스토어 29.6%), ‘질의에 대한 드백 지연’(각각 26.1%, 25.3%) 등이 꼽혔다.

앱을 최초로 등록하기 위해 소요되는 심사기간은 구글 플레이는 등록 시 일 이내(25.6%)에 처리되는 경우가 많았으나, 애플 앱스토어는 6∼7일 이내(42.5%)로 나타났다.

우리나라 국민이 가장 자주 이용하는 앱 마켓은 구글 플레이(67.2%), 애플 스토어(29.7%) 순이었으며, 해당 앱 마켓을 자주 이용하는 이유로는 ‘사용이 편리해서’(67.7%), ‘설치되어 있어서’(61.3%), ‘상품 수가 많아서’(33,5%) 등 으로 나타났다.
"""

# 사용자 프롬프트를 체인에 입력
summary_result = summary_chain.invoke({"text_to_summarize": text_to_summarize})
print("[요약 결과]")
print(summary_result)

[요약 결과]
국내 앱 개발자들은 앱 마켓에서 심사 지연과 등록 거부를 주요 불공정 사례로 경험하며, 인앱 결제의 과도한 수수료를 문제로 인식하고 있다. 2023년 앱 마켓 규모는 감소했으며, 구글 플레이와 애플 앱스토어가 가장 많이 사용된다.


### 롤 토큰 Chat 템플릿

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatPromptTemplate 생성: 시스템/사용자 메시지 정의
chat_template = ChatPromptTemplate([
    ("system", "당신은 질문의 상황에 적절한 인공지능 모델을 찾아주는 AI 전문가입니다"),
    ("user", "다음 상황에 맞는 모델을 찾아줘 \n{input}")
])

# LCEL( LangChain Expression Language )을 이용한 체인 구성
# 프롬프트 → LLM → 문자열 파서 로 이어지는 파이프라인 구성
chat_chain = chat_template | llm | StrOutputParser()

# 체인을 실행할 때는 딕셔너리 형태로 변수(input)을 전달
response = chat_chain.invoke({"input": "우리는 CCTV에 범죄 상황을 인지하기 위한 기능을 넣으려고 한다"})
print("[응답]\n", response)

[응답]
 CCTV에 범죄 상황을 인지하기 위한 기능을 추가하려면, 다음과 같은 인공지능 모델을 고려할 수 있습니다:

1. **객체 탐지 모델**: YOLO (You Only Look Once), SSD (Single Shot MultiBox Detector), Faster R-CNN과 같은 모델을 사용하여 CCTV 영상에서 사람, 차량, 가방 등 특정 객체를 실시간으로 탐지할 수 있습니다.

2. **행동 인식 모델**: LSTM (Long Short-Term Memory) 또는 3D CNN과 같은 모델을 활용하여 사람의 행동을 분석하고, 비정상적인 행동(예: 싸움, 도난 등)을 인식할 수 있습니다.

3. **이상 탐지 모델**: Autoencoder, Isolation Forest와 같은 비지도 학습 모델을 사용하여 정상적인 행동 패턴을 학습하고, 이를 기반으로 이상 행동을 탐지할 수 있습니다.

4. **비디오 분석 플랫폼**: OpenVINO, TensorFlow Object Detection API와 같은 플랫폼을 활용하여 다양한 모델을 통합하고, 실시간으로 CCTV 영상을 분석할 수 있습니다.

이러한 모델들을 조합하여 범죄 상황을 효과적으로 인지하고 경고할 수 있는 시스템을 구축할 수 있습니다.


### 메모리 활용

LLM은 **이전 대화 내용(히스토리)**을 함께 넣어주면 대화를 더 자연스럽게 이어갈 수 있습니다.

LangChain 최신 버전에서는
history라는 자리에 누적된 대화 메시지를 자동으로 넣어주는 기능을 제공합니다.
그래서 같은 세션 ID로 체인을 호출하면, 이전 질문·답변들이 자동으로 기억되고 → 다음 질문에 더 자연스러운 답변을 하게 됩니다.

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1) LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# 2) history + user_input 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 친절한 AI 비서야."),
        MessagesPlaceholder(variable_name="history"),  # 이전 대화가 들어갈 자리
        ("human", "{user_input}"),                     # 새 사용자 입력
    ]
)

In [ ]:
# 3) 기본 체인: 프롬프트 → LLM
base_chain = prompt | llm

# 4) 세션별로 메모리(대화 기록)를 관리할 저장소
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5) RunnableWithMessageHistory로 "메모리 있는 체인" 만들기
llm_chain_with_history = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="user_input",  # 입력에서 어떤 key를 user 메시지로 쓸지
    history_messages_key="history",   # 프롬프트의 MessagesPlaceholder 이름
)


In [ ]:
# 6) 체인 실행 (첫 대화)
response1 = llm_chain_with_history.invoke(
    {"user_input": "BERT 모델에 대하여 간략한 설명 부탁해"},
    config={"configurable": {"session_id": "session-1"}},  # 같은 session_id면 히스토리 공유
)
print("[응답 1]\n", response1.content)

[응답 1]
 BERT(Bidirectional Encoder Representations from Transformers)는 Google이 2018년에 발표한 자연어 처리(NLP) 모델로, 텍스트의 맥락을 이해하기 위해 양방향 Transformer 아키텍처를 사용합니다. BERT는 단어의 의미를 문맥에 따라 파악할 수 있도록 훈련되며, 특히 문맥에서 단어의 의미가 어떻게 변하는지를 반영합니다.

BERT의 주요 특징은 다음과 같습니다:

1. **양방향성**: BERT는 입력 텍스트를 왼쪽과 오른쪽 문맥을 모두 고려하여 처리하므로, 각 단어의 의미를 보다 정확하게 이해할 수 있습니다.

2. **사전 훈련 및 전이 학습**: BERT는 대량의 텍스트 데이터로 사전 훈련된 후, 특정 NLP 작업에 맞게 미세 조정(fine-tuning)될 수 있습니다. 이 접근법은 다양한 자연어 처리 과제를 효과적으로 처리할 수 있는 모델을 만드는 데 유용합니다.

3. **Masked Language Model**: BERT는 일부 단어를 마스킹하고, 모델이 이를 예측하도록 훈련됩니다. 이 과정은 모델이 문맥을 기반으로 단어를 이해하도록 도와줍니다.

4. **다양한 NLP 작업에 적용 가능**: BERT는 질의 응답, 감정 분석, 문서 요약 등 다양한 자연어 처리 작업에서 뛰어난 성능을 보여줍니다.

BERT는 자연어 처리 분야에 큰 영향을 미쳤으며, 이후 다른 많은 모델들이 BERT의 아이디어를 기반으로 발전되었습니다.


In [ ]:
# 7) 체인 실행 (두 번째 대화) - 이전 대화가 history에 자동 저장/사용됨
response2 = llm_chain_with_history.invoke(
    {"user_input": "방금 설명한 모델과 GPT 모델의 차이점은?"},
    config={"configurable": {"session_id": "session-1"}},
)
print("\n[응답 2]\n", response2.content)


[응답 2]
 BERT 모델과 GPT(Generative Pre-trained Transformer) 모델은 모두 Transformer 아키텍처를 기반으로 한 자연어 처리 모델이지만, 몇 가지 중요한 차이점이 있습니다.

### 1. **목적과 용도**
- **BERT**: 주로 자연어 이해(NLU) 작업에 중점을 두고 있으며, 문맥을 이해하고 텍스트를 분석하는 데 강점이 있습니다. 예를 들어, 질문 응답, 감정 분석, 문장 분류 등의 작업에 적합합니다.
- **GPT**: 텍스트 생성(generative) 모델로, 주로 자연어 생성(NLG) 작업에 사용됩니다. 주어진 텍스트에 기반하여 다음 단어나 문장을 생성하는 데 적합합니다. 예를 들어, 대화 생성, 이야기 작성, 문서 요약 등의 작업에 활용됩니다.

### 2. **훈련 방식**
- **BERT**: 양방향으로 문맥을 이해하기 위해 "Masked Language Model"과 "Next Sentence Prediction"이라는 두 가지 태스크로 사전 훈련됩니다. 입력 문장 내에서 일부 단어를 마스킹하고, 모델이 그 단어를 예측하도록 합니다.
- **GPT**: 단방향(왼쪽에서 오른쪽)으로 훈련됩니다. 주어진 단어 시퀀스에서 다음 단어를 예측하는 방식으로 훈련됩니다. 이로 인해 GPT는 자연어 생성에 더 적합합니다.

### 3. **아키텍처**
- **BERT**: Transformer의 인코더(encoder) 부분만 사용합니다. 양방향 처리를 통해 텍스트의 전반적인 맥락을 파악합니다.
- **GPT**: Transformer의 디코더(decoder) 부분만 사용します. 데이터의 흐름이 왼쪽에서 오른쪽으로 진행되며, 이전 단어들만 참고하여 다음 단어를 생성합니다.

### 4. **작업 적용 차이**
- **BERT**: 특정 작업에 맞게 미세 조정(fine-tuning)하여 사용하는 경우가 많습니다.
- **GPT**: 미세 조정 없이도 프롬프트(prompt)를 통해 다양한 작업을 수행할 수 있으며